**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
library("UpSetR")
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Prepare

**Set global variable**

In [2]:
vec = c(
    "fcc_astarr_macs_input_overlap",
    "fcc_astarr_macs_input_union"
)
names(vec) = vec

VEC_TXT_FOLDER = vec
for(txt in vec){cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


In [3]:
TXT_FNAME_ANNOT = "region.annotation.encode_e2g_benchmark.tsv"

**View files**

In [4]:
txt_fdiry = file.path(FD_RES, "region_annotation", "*", "summary")
txt_fname = TXT_FNAME_ANNOT
txt_fglob = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for(txt in vec){cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_overlap/summary/region.annotation.encode_e2g_benchmark.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_union/summary/region.annotation.encode_e2g_benchmark.tsv 


## Import data

**Read region annotation**

In [5]:
### loop to import data
lst = lapply(VEC_TXT_FOLDER, function(txt_folder){
    ### set file directory
    txt_fdiry = file.path(FD_RES, "region_annotation", txt_folder, "summary")
    txt_fname = TXT_FNAME_ANNOT
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### read table
    dat = read_tsv(txt_fpath, show_col_types = FALSE)
    return(dat)
})
names(lst) = VEC_TXT_FOLDER

### assign and show
lst_dat_region_annot_import = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
fun_display_table(head(dat, 3))

$fcc_astarr_macs_input_overlap
[1] 8770   12

$fcc_astarr_macs_input_union
[1] 9229   12



Chrom,ChromStart,ChromEnd,Region,Group,Label,Target,Score,Zcore,NLog10P,Regulated,Source
chr1,3774056,3776283,chr1:3774056-3776283,E2G-Benchmark,Regulated:TRUE,CEP104,-0.2934319,-4.705144,2.395344,TRUE,Ulirsch2016
chr1,3774056,3776283,chr1:3774056-3776283,E2G-Benchmark,Regulated:TRUE,LRRC47,-0.3311781,-5.331209,2.109514,TRUE,Ulirsch2016
chr1,3774056,3776283,chr1:3774056-3776283,E2G-Benchmark,Regulated:TRUE,SMIM1,-0.4720192,-7.667223,3.192703,TRUE,Ulirsch2016


**Read region**

In [6]:
### set file directory
txt_fdiry = file.path(FD_RES, "region", "encode_e2g_benchmark", "summary")
txt_fname = "K562.hg38.ENCODE_E2G.benchmark.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read table
dat = read_tsv(txt_fpath, show_col_types = FALSE)

### assign and show
dat_region_original = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 10375    13


Chrom,ChromStart,ChromEnd,Name,Score,Zcore,Region,Target,NLog10P,Regulated,Source,Group,Label
chr1,3774714,3775214,CEP104|chr1:3691278-3691778:*,-0.2934319,-4.705144,chr1:3774714-3775214,CEP104,2.395344,TRUE,Ulirsch2016,E2G-Benchmark,Regulated:TRUE
chr1,3774714,3775214,LRRC47|chr1:3691278-3691778:*,-0.3311781,-5.331209,chr1:3774714-3775214,LRRC47,2.109514,TRUE,Ulirsch2016,E2G-Benchmark,Regulated:TRUE
chr1,3774714,3775214,SMIM1|chr1:3691278-3691778:*,-0.4720192,-7.667223,chr1:3774714-3775214,SMIM1,3.192703,TRUE,Ulirsch2016,E2G-Benchmark,Regulated:TRUE


## Explore: Count table

**Total links and peaks**

In [7]:
dat = dat_region_original
dat = dat %>% dplyr::mutate(Link = paste(Region, Target, sep = ":"))

### count links (total)
vec = unique(dat$Name)
print(length(vec))
                            
vec = unique(dat$Link)
print(length(vec))

### count region (total)
vec = unique(dat$Region)
print(length(vec))

[1] 10375
[1] 10375
[1] 3949


**Regulated links and peaks**

In [8]:
dat = dat_region_original
dat = dat %>% dplyr::filter(Regulated)
dat = dat %>% dplyr::select(Region, Group, Target) %>% dplyr::distinct()

res = table(dat$Group, dnn = "Group")
dat = as.data.frame(res)

dat_region_link_count = dat
fun_display_table(dat)

Group,Freq
E2G-Benchmark,472


In [9]:
dat = dat_region_original
dat = dat %>% dplyr::filter(Regulated)
dat = dat %>% dplyr::select(Region, Group) %>% dplyr::distinct()

res = table(dat$Group, dnn = "Group")
dat = as.data.frame(res)

dat_region_peak_count = dat
fun_display_table(dat)

Group,Freq
E2G-Benchmark,420


**Region contains regulated links**

In [10]:
lst = lst_dat_region_annot_import
lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::filter(Regulated) %>% 
        dplyr::select(Group, Region, Target) %>%
        dplyr::distinct()
    res = table(dat$Group, dnn = "Group")
    dat = as.data.frame(res)
    return(dat)
})
dat = bind_rows(lst, .id = "Region")

### assign and show
dat_region_annot_count_link = dat
fun_display_table(dat)

Region,Group,Freq
fcc_astarr_macs_input_overlap,E2G-Benchmark,422
fcc_astarr_macs_input_union,E2G-Benchmark,424


In [11]:
lst = lst_dat_region_annot_import
lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::filter(Regulated) %>% 
        dplyr::select(Group, Region) %>%
        dplyr::distinct()
    res = table(dat$Group, dnn = "Group")
    dat = as.data.frame(res)
    return(dat)
})
dat = bind_rows(lst, .id = "Region")

### assign and show
dat_region_annot_count_peak = dat
fun_display_table(dat)

Region,Group,Freq
fcc_astarr_macs_input_overlap,E2G-Benchmark,360
fcc_astarr_macs_input_union,E2G-Benchmark,362


**Sumarize link count**

In [12]:
tmp = dat_region_link_count
tmp = tmp %>% dplyr::mutate(Freq = scales::comma(Freq))
colnames(tmp) = c("Group", "Link (Total)")

dat = dat_region_annot_count_link
dat = dat %>% 
    dplyr::mutate(Region = fun_str_map_atac(Region)) %>%
    dplyr::mutate(Freq   = scales::comma(Freq)) %>%
    tidyr::spread(Region, Freq)

dat = tmp %>% dplyr::right_join(dat, by = "Group")
dat %>% kableExtra::kable("markdown")



|Group         |Link (Total) |ATAC (Overlap) |ATAC (Union) |
|:-------------|:------------|:--------------|:------------|
|E2G-Benchmark |472          |422            |424          |

**Summarize peak count**

In [13]:
tmp = dat_region_peak_count
tmp = tmp %>% dplyr::mutate(Freq = scales::comma(Freq))
colnames(tmp) = c("Group", "Peak (Total)")

dat = dat_region_annot_count_peak
dat = dat %>% 
    dplyr::mutate(Region = fun_str_map_atac(Region)) %>%
    dplyr::mutate(Freq   = scales::comma(Freq)) %>%
    tidyr::spread(Region, Freq)

dat = tmp %>% dplyr::right_join(dat, by = "Group")
dat %>% kableExtra::kable("markdown")



|Group         |Peak (Total) |ATAC (Overlap) |ATAC (Union) |
|:-------------|:------------|:--------------|:------------|
|E2G-Benchmark |420          |360            |362          |